<a href="https://colab.research.google.com/github/Kwasi-Dankwa/f1-prediction-gp/blob/main/CanadianGPPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing FastF1 API
pip install fastf1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.5 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [6]:
# Import relevant libraries
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns


In [11]:
# setting up library to retrieve data
from fastf1 import plotting
import os

# to store downloaded sessions
cache_dir = 'f1_cache' # Defining the cache directory name
if not os.path.exists(cache_dir): # Check if the directory exists
    os.makedirs(cache_dir) # Create the directory if it doesn't exist

# Enabling the cache using the created directory
fastf1.Cache.enable_cache(cache_dir)

# Load the race session
session = fastf1.get_session(2024, 'Canadian Grand Prix', 'R')
session.load()

core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status

In [13]:
# Extracting race results for a single race
results = session.results[['Abbreviation', 'Position', 'TeamName', 'GridPosition', 'Points']]
results = results.rename(columns={
    'Abbreviation': 'driver',
    'TeamName': 'constructor',
    'GridPosition': 'grid',
    'Position': 'position'
})
results['year'] = 2024
results['track'] = 'Canada'

# loop
def get_race_results(year, gp_name='Canadian Grand Prix'):
    try:
        session = fastf1.get_session(year, gp_name, 'R')
        session.load()
        df = session.results[['Abbreviation', 'Position', 'TeamName', 'GridPosition', 'Points']]
        df = df.rename(columns={
            'Abbreviation': 'driver',
            'TeamName': 'constructor',
            'GridPosition': 'grid',
            'Position': 'position'
        })
        df['year'] = year
        df['track'] = 'Canada'
        return df
    except:
        return None

# Build dataset across multiple years
years = list(range(2015, 2025))
all_races = pd.concat([get_race_results(y) for y in years if get_race_results(y) is not None], ignore_index=True)



core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
DEBUG:fastf1.api:Falling back to livetiming mirror (https://livetiming-mirror.fastf1.dev)
logger      WARNING 	Failed to load session info data!
DEBUG:fastf1.fastf1.core:Traceback for failure in session info data
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/fastf1/logger.py", line 151, in __wrapped
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastf1/core.py", line 1470, in _load_session_info
    self._session_info = api.session_info(self.api_path,
                         ^^^^^^^^^^^^^^

In [14]:
# setting target variable
all_races['winner'] = all_races['position'] == 1

# Encoding categoricla variables
from sklearn.preprocessing import LabelEncoder

le_driver = LabelEncoder()
le_constructor = LabelEncoder()

all_races['driver_encoded'] = le_driver.fit_transform(all_races['driver'])
all_races['constructor_encoded'] = le_constructor.fit_transform(all_races['constructor'])



In [15]:
#Building classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = all_races[['grid', 'driver_encoded', 'constructor_encoded', 'year']] #defining feature set
y = all_races['winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.926829268292683


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:44:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
